<a href="https://colab.research.google.com/github/EchenimEdwin/https-github.com-users-EchenimEdwin-/blob/main/Smart_pipeline_monitoring_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Run code in Visual studio to get an interative dashboard
import tkinter as tk
from tkinter import messagebox, filedialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd
from datetime import datetime
import random
import threading
import time


# Enhanced Smart Pipeline System
class SmartPipelineSystem:
    def __init__(self, pressure_threshold, flow_threshold, leak_threshold, humidity_threshold, temperature_threshold):
        self.pressure_threshold = pressure_threshold  # Maximum allowable pressure
        self.flow_threshold = flow_threshold          # Expected flow rate range
        self.leak_threshold = leak_threshold          # Leak detection threshold
        self.humidity_threshold = humidity_threshold  # Maximum allowable humidity
        self.temperature_threshold = temperature_threshold  # Maximum allowable temperature
        self.data = []                                # Stores monitoring data
        self.test_in_progress = False                # Is the system running?
        self.anomaly_detected = False                # Anomaly detection flag
        self.leak_detected = False                   # Leak detection flag
        self.env_issue_detected = False              # Environmental issue flag

    def start_system(self):
        if not self.test_in_progress:
            self.test_in_progress = True
            print("Pipeline monitoring started.")
            self.log_event("System started")

    def stop_system(self):
        if self.test_in_progress:
            self.test_in_progress = False
            print("Pipeline monitoring stopped.")
            self.log_event("System stopped")

    def monitor_pipeline(self, pressure, flow_rate, humidity, temperature):
        """
        Monitors pipeline pressure, flow rate, humidity, and temperature.
        Detects anomalies, leaks, and environmental issues.
        """
        if pressure > self.pressure_threshold:
            self.anomaly_detected = True
            print("High pressure anomaly detected!")
            self.log_event("High pressure anomaly detected")
        elif flow_rate < self.flow_threshold - self.leak_threshold:
            self.leak_detected = True
            print("Leak detected! Flow rate too low.")
            self.log_event("Leak detected")
        elif humidity > self.humidity_threshold:
            self.env_issue_detected = True
            print("High humidity detected!")
            self.log_event("High humidity detected")
        elif temperature > self.temperature_threshold:
            self.env_issue_detected = True
            print("High temperature detected!")
            self.log_event("High temperature detected")
        else:
            self.anomaly_detected = False
            self.leak_detected = False
            self.env_issue_detected = False
            print("Pipeline operating normally.")

        # Log sensor data
        self.data.append({
            "timestamp": datetime.now(),
            "pressure": pressure,
            "flow_rate": flow_rate,
            "humidity": humidity,
            "temperature": temperature
        })

    def export_data_to_excel(self, filename):
        """
        Exports monitoring data to an Excel file.
        """
        df = pd.DataFrame(self.data)
        df.to_excel(filename, index=False)
        print(f"Data exported to {filename}")

    def log_event(self, message):
        """
        Logs significant events with timestamps.
        """
        print(f"[{datetime.now()}] {message}")


# GUI for the Enhanced Smart Pipeline Monitoring System
class PipelineMonitoringGUI:
    def __init__(self, root, system):
        self.root = root
        self.system = system
        self.root.title("Enhanced Smart Pipeline Monitoring System")
        self.running = False  # To control real-time monitoring

        # GUI Components
        self.pressure_label = tk.Label(root, text="Pressure (PSI):")
        self.pressure_label.pack()
        self.pressure_entry = tk.Entry(root)
        self.pressure_entry.pack()

        self.flow_label = tk.Label(root, text="Flow Rate (L/min):")
        self.flow_label.pack()
        self.flow_entry = tk.Entry(root)
        self.flow_entry.pack()

        self.humidity_label = tk.Label(root, text="Humidity (%):")
        self.humidity_label.pack()
        self.humidity_entry = tk.Entry(root)
        self.humidity_entry.pack()

        self.temperature_label = tk.Label(root, text="Temperature (°C):")
        self.temperature_label.pack()
        self.temperature_entry = tk.Entry(root)
        self.temperature_entry.pack()

        self.start_button = tk.Button(root, text="Start Monitoring", command=self.start_monitoring)
        self.start_button.pack()

        self.stop_button = tk.Button(root, text="Stop Monitoring", command=self.stop_monitoring)
        self.stop_button.pack()

        self.export_button = tk.Button(root, text="Export Data to Excel", command=self.export_data)
        self.export_button.pack()

        self.status_label = tk.Label(root, text="System Status: Idle")
        self.status_label.pack()

        # Visualization Plot
        self.figure, self.ax = plt.subplots()
        self.canvas = FigureCanvasTkAgg(self.figure, master=root)
        self.canvas.get_tk_widget().pack()

    def start_monitoring(self):
        """
        Starts the pipeline monitoring process in real time.
        """
        self.system.start_system()
        self.update_status("Monitoring in progress")
        self.running = True
        threading.Thread(target=self.real_time_monitoring, daemon=True).start()

    def stop_monitoring(self):
        """
        Stops the pipeline monitoring process.
        """
        self.system.stop_system()
        self.update_status("Monitoring stopped")
        self.running = False

    def real_time_monitoring(self):
        """
        Simulates real-time data collection and updates the system.
        """
        while self.running:
            # Simulating sensor data (replace this with real IoT sensor inputs)
            pressure = random.uniform(20, 150)  # Simulated pressure in PSI
            flow_rate = random.uniform(50, 100)  # Simulated flow rate in L/min
            humidity = random.uniform(20, 90)  # Simulated humidity in percentage
            temperature = random.uniform(10, 50)  # Simulated temperature in °C

            self.system.monitor_pipeline(pressure, flow_rate, humidity, temperature)
            self.update_plot()
            time.sleep(1)  # Simulate a 1-second data interval

    def export_data(self):
        """
        Prompts the user to save monitoring data to an Excel file.
        """
        filename = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
        if filename:
            self.system.export_data_to_excel(filename)
            messagebox.showinfo("Export Data", f"Data exported successfully to {filename}")

    def update_status(self, status):
        """
        Updates the system status label in the GUI.
        """
        self.status_label.config(text=f"System Status: {status}")

    def update_plot(self):
        """
        Updates the real-time plot with the latest sensor data.
        """
        self.ax.clear()
        if self.system.data:
            df = pd.DataFrame(self.system.data)
            self.ax.plot(df["timestamp"], df["pressure"], label="Pressure (PSI)", color="blue")
            self.ax.plot(df["timestamp"], df["flow_rate"], label="Flow Rate (L/min)", color="green")
            self.ax.plot(df["timestamp"], df["humidity"], label="Humidity (%)", color="orange")
            self.ax.plot(df["timestamp"], df["temperature"], label="Temperature (°C)", color="red")
            self.ax.legend()
            self.ax.set_xlabel("Time")
            self.ax.set_ylabel("Values")
            self.ax.set_title("Pipeline Monitoring: Pressure, Flow Rate, Humidity, Temperature")
            self.canvas.draw()


# Main application
if __name__ == "__main__":
    root = tk.Tk()
    system = SmartPipelineSystem(
        pressure_threshold=120, flow_threshold=70, leak_threshold=10,
        humidity_threshold=80, temperature_threshold=40
    )
    gui = PipelineMonitoringGUI(root, system)
    root.mainloop()

TclError: no display name and no $DISPLAY environment variable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')